# SNIPPET HANDLING

## Imports

In [95]:
# Framework
import torch

# Data handling
import pandas as pd
import numpy as np

# Visualization
import matplotlib.pyplot as plt
import plotly.express as px

# File handling
import os

#Audio manipulation
import torchaudio
from pydub import AudioSegment

In [96]:
print(f'torch: {torch.__version__}')
print(f'torch audio: {torchaudio.__version__}')

torch: 1.13.1
torch audio: 0.13.1


In [97]:
# TEST PATH
folder = '/Users/rblc/code/iamrblc/laica/examples/preprocess_samples'

# LIVE PATH
#folder = '/Users/rblc/code/iamrblc/laica/examples/snippets'

## Creating snippet dataframe

In [98]:
# Define a function to extract the required properties from an audio file
def get_audio_properties(filename):
    waveform, sample_rate = torchaudio.load(filename)
    length = waveform.shape[1] / sample_rate
    num_channels = waveform.shape[0]
    bitrate = torchaudio.info(filename).bits_per_sample

    return length, num_channels, bitrate

In [99]:
# Create an empty pandas dataframe to store the properties
df = pd.DataFrame(columns=["file_name", "length", "num_channels", "bitrate"])

# Iterate over the audio files in the folder and extract their properties
for filename in os.listdir(folder):
    if filename.endswith(".wav"):
        filepath = os.path.join(folder, filename)
        length, num_channels, bitrate = get_audio_properties(filepath)
        df = df.append({"file_name": filename, "length": length, "num_channels": num_channels, "bitrate": bitrate}, ignore_index=True)

FileNotFoundError: [Errno 2] No such file or directory: '/Users/rblc/code/iamrblc/laica/examples/preprocess_samples'

## Checking dataframe properties before audio preprocess

In [ ]:
df.head()

,file_name,length,num_channels,bitrate
0,whine_14124.wav,1.398912,1,16
1,whine_14092.wav,0.600000,1,16
2,whine_14086.wav,0.941043,1,16
3,bark_01283.wav,0.223991,1,16
4,whine_15871.wav,0.400000,1,16


In [ ]:
df.describe()

,length
count,6.000000
mean,1.327324
std,1.562175
min,0.223991
25%,0.450000
50%,0.770522
75%,1.284444
max,4.400000


In [ ]:
df.num_channels.value_counts()

1    4
2    2
Name: num_channels, dtype: int64

In [ ]:
df.bitrate.value_counts()

32    3
16    3
Name: bitrate, dtype: int64

## Audio preprocessing I. - General formatting

In [ ]:
def preprocess(row):
    # Load the audio file
    audio = AudioSegment.from_file(os.path.join(folder, row['file_name']), format="wav")
    
    # Check if 32 bit and convert to 16 bit if necessary
    if audio.sample_width == 4:
        audio = audio.set_sample_width(2)
    
    # Check if stereo and convert to mono if necessary
    if audio.channels == 2:
        audio = audio.set_channels(1)
    
    # Save the preprocessed audio file
    #output_file = os.path.splitext(row['file_name'])[0] + ".wav"
    #audio.export(os.path.join(folder, output_file), format="wav")
    
    output_file = os.path.splitext(row['file_name'])[0] + ".wav"
    output_path = os.path.join(folder, 'snippets_preproc', output_file)
    audio.export(output_path, format="wav")
    return output_path

df['output_file'] = df.apply(preprocess, axis=1)

In [ ]:
df

,file_name,length,num_channels,bitrate,output_file
0,whine_14124.wav,1.398912,1,32,whine_14124
1,whine_14092.wav,0.600000,1,32,whine_14092
2,whine_14086.wav,0.941043,1,32,whine_14086
3,bark_01283.wav,0.223991,1,16,bark_01283
4,whine_15871.wav,0.400000,2,16,whine_15871
5,pant_10388.wav,4.400000,2,16,pant_10388


In [ ]:
# Create an empty pandas dataframe to store the properties
df = pd.DataFrame(columns=["file_name", "length", "num_channels", "bitrate"])

# Iterate over the audio files in the folder and extract their properties
for filename in os.listdir(folder):
    if filename.endswith(".wav"):
        filepath = os.path.join(folder, filename)
        length, num_channels, bitrate = get_audio_properties(filepath)
        df = df.append({"file_name": filename, "length": length, "num_channels": num_channels, "bitrate": bitrate}, ignore_index=True)

/var/folders/x2/6gs4vmmn5bxgqplmvj03kx240000gn/T/ipykernel_2885/3847368972.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({"file_name": filename, "length": length, "num_channels": num_channels, "bitrate": bitrate}, ignore_index=True)
/var/folders/x2/6gs4vmmn5bxgqplmvj03kx240000gn/T/ipykernel_2885/3847368972.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({"file_name": filename, "length": length, "num_channels": num_channels, "bitrate": bitrate}, ignore_index=True)
/var/folders/x2/6gs4vmmn5bxgqplmvj03kx240000gn/T/ipykernel_2885/3847368972.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({"file_name": filename, "length": length, "num_channels": num_channels, "bitrate": 

In [ ]:
df

,file_name,length,num_channels,bitrate
0,whine_14124.wav,1.398912,1,32
1,whine_14092.wav,0.600000,1,32
2,whine_14086.wav,0.941043,1,32
3,bark_01283.wav,0.223991,1,16
4,whine_15871.wav,0.400000,2,16
5,pant_10388.wav,4.400000,2,16
